In [1]:
import sqlite3
import logging
import pandas as pd
from urllib.parse import urlparse
from tld import get_tld, get_fld

In [2]:
logger = logging.getLogger('analysis')
logger.setLevel('INFO')

In [3]:
# Specify your local path to the sqlite db containing crawl data
SQLITE_LOCAL_PATH = '/Users/liam/dev/OpenWPM/data/gemeente_social_complete.sqlite'

# Specify the list for scoial media FLDs that we are interested in observing
SM_FLDS = ['facebook', 'instagram', 'linkedin', 'google', 'youtube', 'twitter']
#, 'whatsapp', 'snapchat', 'amazon', 'reddit']

In [4]:
# Read sqlite query results into a pandas DataFrame
con = sqlite3.connect(SQLITE_LOCAL_PATH)

In [5]:
cursor = con.cursor()
# Parse javascript table from SQLite
javascript = pd.read_sql_query("SELECT * from javascript", con)
# Parse SITE_VISITS table from SQLite
site_visits = pd.read_sql_query("SELECT * from site_visits", con)

In [6]:
# Helper function to get TLD+1 level of crawled link.
def parse_tld_url(url):
    return get_fld(url, fail_silently=True)

# Helper function to match social media names in url FLDs
def sm_match(url, sm_name):
    if url is not None:
        return sm_name in url
    else:
        return None

In [7]:
# Extract the First level domain for page links at the top level;
# pages linked to via a hyperlink from the first visible page when visiting a gemeente website
site_visits['tld_url'] = site_visits['site_url'].apply(parse_tld_url)

# Extract the top level domain from the keyed gemeente website
site_visits['tld_parent'] = site_visits['parent_url'].apply(parse_tld_url)

# Create a boolean column for each social media site fo interest.
for i in SM_FLDS:
    site_visits[('is_' + i)] = site_visits['tld_url'].apply(sm_match, args=(i,))
    
# Return social media aggregates per gemeente per social media platform.
social_media_pressence = site_visits[['parent_url']+ [('is_' + i) for i in SM_FLDS]]\
    .groupby(['parent_url']).agg(['sum'])\
    .applymap(lambda x: True if x >= 1 else False)
social_media_pressence.head()

,is_facebook,is_instagram,is_linkedin,is_google,is_youtube,is_twitter
,sum,sum,sum,sum,sum,sum
parent_url,,,,,,
http://gemeente.groningen.nl,True,True,False,False,False,True
http://www.aaenhunze.nl,True,False,False,False,True,True
http://www.aalsmeer.nl,True,False,True,False,True,True
http://www.aalten.nl,True,False,True,False,False,True
http://www.achtkarspelen.nl,True,False,False,False,False,True


In [8]:
social_media_pressence.sum()/len(site_visits['parent_url'].unique())*100
#num_gemeente = len(site_visits['parent_url'].unique())
#javascript.head()
#con.close()

is_facebook   sum    81.126761
is_instagram  sum    33.521127
is_linkedin   sum    23.943662
is_google     sum     8.450704
is_youtube    sum    27.042254
is_twitter    sum    83.380282
dtype: float64

In [9]:
counted_total = round(\
    social_media_pressence.sum(axis=1).apply(lambda p: p > 0).sum()\
                      /len(site_visits['parent_url'].unique()),  3) * 100
print(str(counted_total) + '% of gemeente websites link directly to at least one of the 6 most popular social network pages on their first page.')

84.8% of gemeente websites link directly to at least one of the 6 most popular social network pages on their first page.


In [10]:
# Display the full, untruncated social media presence data frame.
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 2000)
    pd.set_option('display.float_format', '{:20,.2f}'.format)
    pd.set_option('display.max_colwidth', -1)
    print(x)
    pd.reset_option('display.max_rows')
    pd.reset_option('display.max_columns')
    pd.reset_option('display.width')
    pd.reset_option('display.float_format')
    pd.reset_option('display.max_colwidth')
    
print_full(social_media_pressence)

                                      is_facebook is_instagram is_linkedin is_google is_youtube is_twitter
                                              sum          sum         sum       sum        sum        sum
parent_url                                                                                                
http://gemeente.groningen.nl           True        True         False       False     False      True     
http://www.aaenhunze.nl                True        False        False       False     True       True     
http://www.aalsmeer.nl                 True        False        True        False     True       True     
http://www.aalten.nl                   True        False        True        False     False      True     
http://www.achtkarspelen.nl            True        False        False       False     False      True     
http://www.alblasserdam.nl             False       False        False       False     False      False    
http://www.alkmaar.nl                

In [11]:
round(social_media_pressence.sum(axis=1)).sort_values(ascending=False)

parent_url
http://www.oldenzaal.nl               6
http://www.tubbergen.nl               6
http://www.heerlen.nl                 6
http://www.peelenmaas.nl              6
http://www.helmond.nl                 5
http://www.zevenaar.nl                5
http://www.lelystad.nl                5
http://www.krimpenerwaard.nl          5
http://www.hollandskroon.nl           5
http://www.hilversum.nl               5
http://www.wijchen.nl                 5
http://www.delft.nl                   5
http://www.heuvelrug.nl               5
http://www.hengelo.nl                 5
http://www.stichtsevecht.nl           5
http://www.dinkelland.nl              5
http://www.zuidplas.nl                5
http://www.weert.nl                   5
http://www.teylingen.nl               5
http://www.hardenberg.nl              5
http://www.ede.nl                     5
http://www.gemeentestein.nl           5
http://www.ermelo.nl                  5
http://www.etten-leur.nl              5
http://www.leudal.nl         